<div class="head0">
    <div class="head0__name">
        TMP file
    </div>
</div>

In [1]:
import sys
import time

import numpy as np
import scipy.sparse as sp
from scipy.integrate import solve_ivp
from scipy.linalg import expm as expm
import matplotlib.pyplot as plt

sys.path.extend(['./../lib', './../helpers'])
from intertrain import Intertrain
from solver import Solver as Solver

from helpers import init_jupyter; init_jupyter()

Start | 12:03PM MSK on Sep 07, 2019 |
-------------------------------------


<div class="head1">
    <div class="head1__name">
        Tests for solver in the TT-format
    </div>
</div>

In [3]:
s = 10.
Dc = 1.

def func_f0(x, t):
    return np.zeros(x.shape)

def func_f1(x, t):
    return np.zeros(x.shape)

def func_r0(x):
    r = np.exp(-1. * np.sum(x*x, axis=0) / (2. * s))
    r/= np.sqrt(2. * np.pi * s)
    return r.reshape(-1)

def func_rt(x, t):
    r = np.exp(-1. * np.sum(x*x, axis=0) / (2. * s + 4. * Dc * t))
    r/= np.sqrt(2. * np.pi * s + 4. * np.pi * Dc * t)
    return r.reshape(-1)

def func_rs(x):
    return np.zeros(x.shape[1])

t_poi = 10
t_min = 0.
t_max = 10.

x_poi = 15
x_min = -5.
x_max = +5.

SL = Solver(d=3, with_tt=True)
SL.set_grid_t(t_poi, t_min, t_max, t_hst=10)
SL.set_grid_x(x_poi, x_min, x_max)
SL.set_funcs(func_f0, func_f1, func_r0, func_rt, func_rs)
SL.set_coefs(Dc)

SL.prep()
SL.calc()


Solve:  22%|▏| 2/9 [00:07<00:26,  3.84s/step, | At T=2.22e+00 : n=5.52e-02 e=9.80e-01 es=9.80e-01]

ZeroDivisionError: float division by zero

<div class="head1">
    <div class="head1__name">
        Various
    </div>
</div>

In [7]:
n = [2, 3, 4]
N = np.prod(n)
A = np.arange(N*N).reshape(N, N)
i = np.array([
    [0, 1, 1, 0],
    [1, 2, 2, 0],
    [2, 3, 2, 1]
])
print(np.unravel_index(8, n, order='F'))

k = np.ravel_multi_index(i, n, order='F')
print(k)
A[np.ix_(k, k)]

(0, 1, 1)
[14 23 17  6]


array([[350, 359, 353, 342],
       [566, 575, 569, 558],
       [422, 431, 425, 414],
       [158, 167, 161, 150]])

<div class="end"></div>